In [1]:
!sudo apt update

[sudo] password for dhpcsap: 

In [ ]:
!sudo apt purge *nvidia* -y

Streaming output truncated to the last 5000 lines.
Package 'linux-objects-nvidia-450-server-5.15.0-1049-gcp' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1049-intel-iotg' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1049-oracle' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-105-generic' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-105-lowlatency' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-aws' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-azure' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-intel-iotg' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1050-oracle' is not installed, so not removed
Package 'linux-objects-nvidia-450-server-5.15.0-1051-aws' is not installed, so not removed
Package 

In [ ]:
!sudo apt install nvidia-driver-530 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  ocl-icd-opencl-dev opencl-c-headers opencl-clhpp-headers
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  cpp-12 dctrl-tools dkms fakeroot gcc-12 keyboard-configuration libasan8
  libfakeroot libfontenc1 libgcc-12-dev libjansson4 liblocale-gettext-perl
  libnvidia-cfg1-555 libnvidia-common-555 libnvidia-compute-555
  libnvidia-decode-555 libnvidia-encode-555 libnvidia-extra-555
  libnvidia-fbc1-555 libnvidia-gl-555 libtsan2 libudev1 libxcvt0 libxfont2
  libxkbfile1 nvidia-compute-utils-555 nvidia-dkms-555 nvidia-driver-555
  nvidia-firmware-555-555.42.02 nvidia-kernel-common-555
  nvidia-kernel-source-555 nvidia-prime nvidia-settings nvidia-utils-555
  python3-xkit screen-resolution-extra systemd-hwe-hwdb udev x11-xkb-utils
  xcvt xfonts-base xfonts-encodings xfo

In [ ]:
!clinfo

Number of platforms                               0


In [ ]:
!apt-get install opencl-headers

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  ocl-icd-opencl-dev
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  opencl-headers
0 upgraded, 1 newly installed, 0 to remove and 44 not upgraded.
Need to get 1,754 B of archives.
After this operation, 12.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 opencl-headers all 3.0~2022.01.04-1 [1,754 B]
Fetched 1,754 B in 0s (4,583 B/s)
Selecting previously unselected package opencl-headers.
(Reading database ... 123831 files and directories currently installed.)
Preparing to unpack .../opencl-headers_3.0~2022.01.04-1_all.deb ...
Unpacking opencl-headers (3.0~2022.01.04-1) ...
Setting up opencl-headers (3.0~2022.01.04-1) ...


In [ ]:
!apt-get install ocl-icd-libopencl1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ocl-icd-libopencl1 is already the newest version (2.2.14-3).
ocl-icd-libopencl1 set to manually installed.
The following package was automatically installed and is no longer required:
  ocl-icd-opencl-dev
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
!sudo find /usr/ -name cl.h 2>/dev/null

/usr/include/CL/cl.h
/usr/local/lib/python3.10/dist-packages/tensorflow/include/third_party/gpus/cuda/include/CL/cl.h
/usr/local/cuda-12.2/targets/x86_64-linux/include/CL/cl.h


In [ ]:
%%writefile addition_of_two_integers.c
#include <stdio.h>
#include <stdlib.h>
#include <CL/cl.h>

#define CL_USE_DEPRECATED_OPENCL_1_1_APIS


const char *kernelSource =
    "__kernel void arrayAdd(__global int *a, __global int *b, __global int *c) {\n"
    "   int i = get_global_id(0);\n"
    "   c[i] = a[i] + b[i];\n"
    "}\n";

int main() {
    cl_platform_id platform;
    cl_device_id device;
    cl_context context;
    cl_command_queue queue;
    cl_program program;
    cl_kernel kernel;
    cl_mem bufferA, bufferB, bufferC;
    cl_event event;
    cl_int err;

    // Initialize inputs
    int *a = (int *)malloc(sizeof(int));
    int *b = (int *)malloc(sizeof(int));
    int *c = (int *)malloc(sizeof(int));

    *a = 222;
    *b = 333;

    // Step 1: Create OpenCL context and command queue
    clGetPlatformIDs(1, &platform, NULL);
    clGetDeviceIDs(platform, CL_DEVICE_TYPE_GPU, 1, &device, NULL);
    context = clCreateContext(NULL, 1, &device, NULL, NULL, NULL);
    queue = clCreateCommandQueue(context,device, CL_QUEUE_PROFILING_ENABLE, NULL);

    // Step 2: Create and build OpenCL program
    program = clCreateProgramWithSource(context, 1, (const char **)&kernelSource, NULL, NULL);
    clBuildProgram(program, 1, &device, NULL, NULL, NULL);

    // Step 3: Create kernel
    kernel = clCreateKernel(program, "arrayAdd", NULL);

    // Step 4: Create memory buffer
    bufferA = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, sizeof(int), a, NULL);
    bufferB = clCreateBuffer(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, sizeof(int), b, NULL);
    bufferC = clCreateBuffer(context, CL_MEM_WRITE_ONLY, sizeof(int), NULL, NULL);

    // Step 5: Set kernel arguments
    clSetKernelArg(kernel, 0, sizeof(cl_mem), &bufferA);
    clSetKernelArg(kernel, 1, sizeof(cl_mem), &bufferB);
    clSetKernelArg(kernel, 2, sizeof(cl_mem), &bufferC);

    // Step 6: Enqueue kernel for execution and measure time
    size_t globalSize = 1;
    // clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, NULL, 0, NULL, &event);

    err = clEnqueueNDRangeKernel(queue, kernel, 1, NULL, &globalSize, NULL, 0, NULL, &event);
    if (err != CL_SUCCESS)
    {
        // Handle the error here, e.g., print an error message and exit
        printf("Error enqueueing kernel: %d\n", err);
        return 1;
    }

    clWaitForEvents(1, &event);

    // Get kernel execution time
    cl_ulong startTime, endTime;
    clGetEventProfilingInfo(event, CL_PROFILING_COMMAND_START, sizeof(cl_ulong), &startTime, NULL);
    clGetEventProfilingInfo(event, CL_PROFILING_COMMAND_END, sizeof(cl_ulong), &endTime, NULL);
    double executionTime = (double)(endTime - startTime) / 1000000.0;   // Convert nanoseconds to milliseconds

    // Step 7: Read the result from the device
    clEnqueueReadBuffer(queue, bufferC, CL_TRUE, 0, sizeof(int), c, 0, NULL, NULL);

    // Step 8: Print the result and execution time
    printf("Execution Time: %.6f milliseconds\n", executionTime);
    printf("Result: ");
    printf("%d", *c);
    printf("\n");

    // Step 9: Release resources
    clReleaseMemObject(bufferA);
    clReleaseMemObject(bufferB);
    clReleaseMemObject(bufferC);
    clReleaseKernel(kernel);
    clReleaseProgram(program);
    clReleaseCommandQueue(queue);

    // Free allocated memory
    free(a);
    free(b);
    free(c);

    return 0;


}

Overwriting addition_of_two_integers.c


In [ ]:
!gcc -o addition_of_two_integers addition_of_two_integers.c -I "/usr/include/CL" -L "/usr/include/CL" -lOpenCL

In file included from /usr/include/CL/cl.h:20,
                 from addition_of_two_integers.c:3:
/usr/include/CL/cl_version.h:22:9: note: ‘#pragma message: cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)’
   22 | #pragma message("cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)")
      |         ^~~~~~~
addition_of_two_integers.c: In function ‘main’:
addition_of_two_integers.c:37:5: warning: ‘clCreateCommandQueue’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
   37 |     queue = clCreateCommandQueue(context,device, CL_QUEUE_PROFILING_ENABLE, NULL);
      |     ^~~~~
In file included from addition_of_two_integers.c:3:
/usr/include/CL/cl.h:1906:1: note: declared here
 1906 | clCreateCommandQueue(cl_context                     context,
      | ^~~~~~~~~~~~~~~~~~~~


In [ ]:
!./addition_of_two_integers

Error enqueueing kernel: -36
